<a href="https://colab.research.google.com/github/RamiAmasha31/ABM-applied_mathmatics/blob/main/Multiple_runs_multiple_params_ABM_Applied_Mathematics_interactive_graphs%2BnormalDistrub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Final Project - Applied Mathematics**
# The following notebook is the code for simulation.

Downloading dependencies

In [15]:
!pip install ipywidgets
!pip install matplotlib


The Agent class is defined as follow:

In [16]:
class Agent:
    def __init__(self, infected, immune, remaining_days_of_infection, remaining_days_of_immunity):
        self.infected = infected
        self.immune = immune
        self.remaining_days_of_infection = remaining_days_of_infection
        self.remaining_days_of_immunity = remaining_days_of_immunity
        self.susceptible=True

The following function for picking two rando agents.

In [17]:
import random
def PickTwoAgentsWithContacts(numAgents):
    """
    Randomly selects two different agent indices from the range [0, numAgents-1]
    with a probability of being in contact.

    Parameters:
    - numAgents (int): The total number of agents.
    - contact_prob (float): The probability of two agents being in contact.

    Returns:
    - tuple: A tuple containing two distinct agent indices.
    """
    # Randomly select two initial indices
    index1 = random.randint(0, numAgents - 1)
    index2 = random.randint(0, numAgents - 1)

    # Ensure the selected indices are different
    while index1 == index2 :
        index1 = random.randint(0, numAgents - 1)
        index2 = random.randint(0, numAgents - 1)

    return index1, index2

The following function for initilaizing the agents array

In [18]:
import numpy as np
import random

def start_step(agents,number_of_agents,infected_at_start,immunity_duartion,infection_duration):
  if infected_at_start  > number_of_agents:
            raise ValueError("Number of initially infected agents cannot exceed the total number of agents.")
 ## create agents
  for i in range(0,number_of_agents):
    agents.append(Agent(False,False,np.random.normal(180,10,1),np.random.normal(60,10,1)))
  ## infect random subset at the start
  infected_indices = random.sample(range(number_of_agents), infected_at_start)
  for index in infected_indices:
        agents[index].infected = True
        agents[index].remaining_days_of_infection = np.random.normal(60,10,1)
        agents[index].susceptible=False
  return agents

The following code for infecting agent

In [19]:
import random

def infect(agent1,agent2,infection_probability,infection_duration):
    random_number = random.random()
    # If agent1 is not infected
    if not agent1.infected and agent2.infected and not agent1.immune and random_number < infection_probability:
        agent1.infected = True
        agent1.remaining_days_of_infection = np.random.normal(60,10,1)
        #overall_infected += 1
        ##susceptibles-=1
        agent1.susceptible=False
    # If agent2 is not infected
    elif not agent2.infected and agent1.infected and not agent2.immune and random_number < infection_probability:
        agent2.infected = True
        agent2.remaining_days_of_infection = np.random.normal(60,10,1)
        #overall_infected += 1
        agent2.susceptible=False

        ##susceptibles-=1
    return agent1, agent2

The following code for death and birth

In [20]:
import numpy as np

def Birth_death(agents, birth_pulse, birth_rate_yearly, infected_per_birth_duration, birth_interval, birth_duration, immunity_duration, infection_duration, new_born, deads,N):
    num_new_agents = 0
    num_dead_agents = 0
    b = 0.1
    m=(birth_rate_yearly-b)/int(N)



    # Check if the birth pulse matches the birth interval
    if birth_pulse % birth_interval < birth_duration:
        # Calculate the expected number of new agents over the birth duration
        expected_new_agents_over_duration = birth_rate_yearly * len(agents)

        # Generate the number of new agents using Poisson distribution
        num_new_agents = np.random.poisson(expected_new_agents_over_duration/birth_duration)

        # Create new agents with the specified parameters
        agents = create_new_agents(agents, num_new_agents, infected_per_birth_duration, immunity_duration, infection_duration)
        #new_born.append(num_new_agents)

    # Calculate daily death rate
    death_rate = m * len(agents) + b
    #daily_death_rate = 1 - pow((1 - death_rate), 1 / birth_interval)
    daily_death_rate=death_rate/birth_interval
    #print(daily_death_rate)

    # Generate the number of dead agents using binomial distribution
    num_dead_agents = np.random.binomial(len(agents), daily_death_rate)

    # Remove dead agents
    agents = remove_dead_agents(agents, num_dead_agents)

    return agents, num_new_agents, num_dead_agents

The following code for revmoving dead agents

In [21]:
def remove_dead_agents(agents, num_dead_agents):
    global NumberOfInfected
    if num_dead_agents >= len(agents):
        agents.clear()
    else:
        for _ in range(num_dead_agents):
            index = random.randint(0, len(agents) - 1)
            agents.pop(index)
    return agents

The following code for creating new agents with specifec number of infected agents

In [22]:
def create_new_agents(agents, num_new_agents,infected_per_birth_duration, immunity_duration, infection_duration):
    number=0
    if infected_per_birth_duration > len(agents):
        number = len(agents)
    else :
      number=infected_per_birth_duration

    for i in range(num_new_agents):
        if i < number:
            agents.append(Agent(True,False,np.random.normal(180,10,1),np.random.normal(60,10,1)))

        else:
            agents.append(Agent(False,False,np.random.normal(180,10,1),np.random.normal(60,10,1)))

    return agents

The following code is for updating the parameters after each simulation step

In [23]:
def update_params(agents, immunity_duration,Overall_infected,susceptibles):
    for i in range(len(agents)):
        if agents[i].infected:
            # If the agent is infected
            agents[i].remaining_days_of_infection -= 1

            if agents[i].remaining_days_of_infection == 0:
                # The agent has ended the duration of infection and started the immunity duration
                agents[i].infected = False
                agents[i].immune = True
                agents[i].remaining_days_of_immunity = np.random.normal(180,10,1)
                #Overall_infected -= 1

        else:
            if agents[i].immune:
                # The agent is immune
                agents[i].remaining_days_of_immunity -= 1

                if agents[i].remaining_days_of_immunity == 0:
                    # The agent has ended the immunity duration
                    agents[i].immune = False
                   # susceptibles+=1
                    agents[i].susceptible=True
    Overall_infected=sum(agent.infected == True for agent in agents)

    susceptibles=sum(agent.susceptible == True for agent in agents)
    return agents,Overall_infected,susceptibles

The following code is the main simulation

In [24]:
import os
import pandas as pd
import matplotlib.pyplot as plt
def simulation_run(number_of_agents,simualtion_duration,birth_rate_yearly,birth_duration,infected_at_start,R0_,N_):
  ##parameters
  infection_duration=60
  immunity_duration=180
  number_of_agents=number_of_agents
  simualtion_duration= simualtion_duration
  birth_rate_yearly=birth_rate_yearly
  birth_interval=365
  birth_duration=birth_duration
  infected_at_start=infected_at_start
  new_born=[0]* int(simualtion_duration)
  infected=[0]* int(simualtion_duration)
  deads=[0]* int(simualtion_duration)
  birth_pulse=0
  susceptibles= [0]* int(simualtion_duration)
  sus=0
  infected=0
  num_new_birhths=0
  overALL_infected=[0]* int(simualtion_duration)
  ##
  infected_per_birth_duration=0
  number_of_interactions_per_day_per_agent=1/10
  R0=R0_
  TMP=infection_duration*number_of_interactions_per_day_per_agent
  infection_probability=float(R0)/TMP
  N=N_
  overall_infected=0
  ##

  agents=[]
  agents=start_step(agents,number_of_agents,infected_at_start,immunity_duration,infection_duration)
  overALL_infected[0]=(infected_at_start)
  susceptibles.append(len(agents)-infected_at_start)
  number_of_interactions_per_day = int(len(agents) *number_of_interactions_per_day_per_agent // 2)



  for i in range(simualtion_duration):
    for j in range(number_of_interactions_per_day):
      index1, index2 = PickTwoAgentsWithContacts(len(agents))
      agent1, agent2=infect(agents[index1],agents[index2],infection_probability,infection_duration)
    birth_pulse += 1
    agents, num_dead_agents,num_new_birhths=Birth_death(agents,birth_pulse,birth_rate_yearly,infected_per_birth_duration,birth_interval,birth_duration,immunity_duration,infection_duration,new_born,deads,N)
    deads[i]=(num_dead_agents)
    new_born[i]=num_new_birhths
    agents,infected,sus=update_params(agents, immunity_duration,infected,sus)
    susceptibles[i]=(sus)
    overALL_infected[i]=(infected)

  return overALL_infected,susceptibles,new_born,deads



In [25]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd

def run_simulation_with_params( number_of_agents, simualtion_duration, birth_rate_yearly, birth_duration, infected_at_start, R0_, N_):
    results = simulation_run(number_of_agents, simualtion_duration, birth_rate_yearly, birth_duration, infected_at_start, R0_, N_)
    plot_simulation_results(results)
    display_parameter_values( number_of_agents, simualtion_duration, birth_rate_yearly, birth_duration, infected_at_start, R0_, N_)

def plot_simulation_results(results):
    overALL_infected, susceptibles, new_born, deads = results
    plt.figure(figsize=(12, 8))
    plt.plot(overALL_infected, label='Infected')
    plt.xlabel('Time')
    plt.ylabel('Number of Infected Agents')
    plt.title('Number of Infected Agents Over Time')
    plt.legend()
    plt.grid(True)
    plt.show()

def display_parameter_values( number_of_agents, simualtion_duration, birth_rate_yearly, birth_duration, infected_at_start, R0_, N_):
    params = {
        'Number of Agents': number_of_agents,
        'Simulation Duration': simualtion_duration,
        'Birth Rate Yearly': birth_rate_yearly,
        'Birth Duration': birth_duration,
        'Infected at Start': infected_at_start,
        'R0': R0_,
        'N': N_
    }

    changed_param = None
    for slider in sliders:
        if slider.value != slider.default_value:
            changed_param = slider.description

    html = "<h3>Current Parameter Values</h3><ul>"
    for key, value in params.items():
        if key == changed_param:
            html += f"<li><strong>{key}: {value} (changed)</strong></li>"
        else:
            html += f"<li>{key}: {value}</li>"
    html += "</ul>"
    display(HTML(html))

# Define the interactive widgets

number_of_agents_slider = widgets.IntSlider(value=500, min=100, max=1000, step=50, description='Number of Agents')
simualtion_duration_slider = widgets.IntSlider(value=30000, min=5000, max=50000, step=5000, description='Simulation Duration')
birth_rate_yearly_slider = widgets.FloatSlider(value=0.3, min=0.1, max=1.0, step=0.1, description='Birth Rate Yearly')
birth_duration_slider = widgets.IntSlider(value=365, min=100, max=500, step=50, description='Birth Duration')
infected_at_start_slider = widgets.IntSlider(value=5, min=1, max=20, step=1, description='Infected at Start')
R0_slider = widgets.FloatSlider(value=2.0, min=0.5, max=5.0, step=0.5, description='R0')
N_slider = widgets.IntSlider(value=500, min=100, max=1000, step=50, description='N')

# List of sliders for checking changed values
sliders = [

    number_of_agents_slider,
    simualtion_duration_slider,
    birth_rate_yearly_slider,
    birth_duration_slider,
    infected_at_start_slider,
    R0_slider,
    N_slider
]

# Save default values to sliders
for slider in sliders:
    slider.default_value = slider.value

# Define the interactive button
run_button = widgets.Button(description='Run Simulation')

# Function to update and run the simulation
def on_run_button_clicked(b):

    number_of_agents = number_of_agents_slider.value
    simualtion_duration = simualtion_duration_slider.value
    birth_rate_yearly = birth_rate_yearly_slider.value
    birth_duration = birth_duration_slider.value
    infected_at_start = infected_at_start_slider.value
    R0_ = R0_slider.value
    N_ = N_slider.value
    run_simulation_with_params( number_of_agents, simualtion_duration, birth_rate_yearly, birth_duration, infected_at_start, R0_, N_)

run_button.on_click(on_run_button_clicked)

# Create a container for the widgets and the plot
left_box = widgets.VBox([number_of_agents_slider, simualtion_duration_slider, birth_rate_yearly_slider, birth_duration_slider, infected_at_start_slider, R0_slider, N_slider, run_button])
right_box = widgets.Output()

# Display the widgets and the plot side by side
display(widgets.HBox([left_box, right_box]))
